In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [33]:
# Dependencies
from pymongo import MongoClient
import os
import pandas as pd

from urls_list import * #where all urls and paths are saved
from config import *

### Check DB for any duplicate rental IDs

In [28]:
def checkSanity(collection, pk):
    client = MongoClient(db_connection_string)
    
    records = list(client.ETLInsights[collection].find({}, {pk:1, '_id':0}))
    total_unique_records = len(set(post[pk] for post in records))
    total_records = client.ETLInsights[collection].count_documents({})
    print(f"Total records: {total_records}")
    print(f"Total uniique records: {total_unique_records}")
    if(total_unique_records == total_records):
        print(f"{collection} is sane")
    else:
        print(f"Sanity check failed for {collection}")
    client.close()

In [29]:
checkSanity("HistoricRental", "id")

Total records: 3315
Total uniique records: 3315
HistoricRental is sane


In [30]:
checkSanity("CurrentRental", "id")

Total records: 2938
Total uniique records: 2938
CurrentRental is sane


### Insert Kijiji Data to the HistoricRental DB

In [34]:
Kijiji = pd.read_csv(os.path.join('..', 'Data', 'kijiji_rental_final.csv'))

In [37]:
client = MongoClient(db_connection_string)
client.ETLInsights.HistoricRental.insert_many(Kijiji.T.to_dict().values())
client.close()

In [38]:
checkSanity("HistoricRental", "id")

Total records: 3837
Total uniique records: 3837
HistoricRental is sane


### Insert Community assets Data in DB